Density analysis along lines connecting cylinder centers in hexagonally packed micelle system.
Creates 1D density profiles along 6 lines (±60° to x-axis) and 3 z-axis lines passing through first 3 cylinder COMs.

In [4]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda
import warnings
warnings.filterwarnings('ignore')
from scipy.ndimage import gaussian_filter1d
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))
from md_styler import MDStyler

In [82]:
def unwrap_cylinder_atoms(universe, start_idx, end_idx, box_dims):
    """
    Unwrap atoms belonging to one cylinder to handle PBC properly.
    """
    atoms = universe.atoms[start_idx:end_idx+1]
    positions = atoms.positions.copy()
    
    # Use first atom as reference
    ref_pos = positions[0]
    
    for i in range(1, len(positions)):
        # Check each dimension
        for dim in range(3):
            diff = positions[i, dim] - ref_pos[dim]
            if diff > box_dims[dim]/2:
                positions[i, dim] -= box_dims[dim]
            elif diff < -box_dims[dim]/2:
                positions[i, dim] += box_dims[dim]
    
    return positions

def calculate_cylinder_coms(universe, n_cylinders=15, atoms_per_cylinder=6160):
    """
    Calculate center of mass for each cylinder, handling PBC.
    """
    box_dims = universe.dimensions[:3]
    coms = []
    
    print(f"Calculating COMs for {n_cylinders} cylinders...")
    
    for i in range(n_cylinders):
        start_idx = i * atoms_per_cylinder
        end_idx = start_idx + atoms_per_cylinder - 1
        
        # Unwrap cylinder atoms
        unwrapped_pos = unwrap_cylinder_atoms(universe, start_idx, end_idx, box_dims)
        
        # Calculate COM
        com = np.mean(unwrapped_pos, axis=0)
        
        # Wrap COM back into box
        com = com % box_dims
        coms.append(com)
        
        print(f"  Cylinder {i}: COM at ({com[0]:.2f}, {com[1]:.2f}, {com[2]:.2f})")
    
    return np.array(coms)

def define_analysis_lines(coms, extend_length=5.0):
    """
    Define 9 analysis lines total:
    - 6 lines: 2 lines per first 3 cylinder COMs at ±60° to x-axis (325 Å long)
    - 3 lines: 1 line per first 3 cylinder COMs along z-axis (shorter, same z-range)
    """
    lines_angular = []
    lines_z = []
    line_length = 375 + 2*(extend_length-5)*10
    # Angular lines (same as before)
    angles = [np.pi/3 + np.pi, -np.pi/3]  # +240° and +120°
    
    # Find the maximum z-coordinate among first 3 cylinders
    max_z = max(coms[i][2] for i in range(3))
    start_z = max_z + extend_length * 10  # Convert nm to Å
    
    for i in range(3):  # First 3 cylinders
        com = coms[i]
        
        for angle in angles:
            # Direction vector
            direction = np.array([np.cos(angle), 0, np.sin(angle)])
            
            # Calculate start point
            t_to_com = (com[2] - start_z) / direction[2]
            start_x = com[0] - t_to_com * direction[0]
            
            # Define start and end points
            start_point = np.array([start_x, com[1], start_z])
            end_point = start_point + line_length * direction
            
            lines_angular.append({
                'cylinder_idx': i,
                'angle_deg': np.degrees(angle),
                'direction': direction,
                'start': start_point,
                'end': end_point,
                'center': com,
                'length': line_length,
                'type': 'angular'
            })
            
            print(f"Angular Line {len(lines_angular)}: Cylinder {i}, {np.degrees(angle):+.0f}° to x-axis")
            print(f"  Start: ({start_point[0]:.1f}, {start_point[1]:.1f}, {start_point[2]:.1f})")
            print(f"  End: ({end_point[0]:.1f}, {end_point[1]:.1f}, {end_point[2]:.1f})")
            print(f"  Length: {line_length:.1f} Å")
    
    # Z-axis lines (new)
    # Find the z-range from angular lines
    all_z_coords = []
    for line in lines_angular:
        all_z_coords.extend([line['start'][2], line['end'][2]])
    
    z_min = min(all_z_coords)
    z_max = max(all_z_coords)
    z_line_length = z_max - z_min
    
    print(f"\nZ-axis lines will span from z={z_min:.1f} to z={z_max:.1f} (length: {z_line_length:.1f} Å)")
    
    for i in range(3):  # First 3 cylinders
        com = coms[i]
        
        # Direction vector (straight down in z)
        direction = np.array([0, 0, -1])  # -π/2 direction
        
        # Start and end points
        start_point = np.array([com[0], com[1], z_max])
        end_point = np.array([com[0], com[1], z_min])
        
        lines_z.append({
            'cylinder_idx': i,
            'angle_deg': -90,
            'direction': direction,
            'start': start_point,
            'end': end_point,
            'center': com,
            'length': z_line_length,
            'type': 'z_axis'
        })
        
        print(f"Z-axis Line {i+1}: Cylinder {i}, -90° (z-direction)")
        print(f"  Start: ({start_point[0]:.1f}, {start_point[1]:.1f}, {start_point[2]:.1f})")
        print(f"  End: ({end_point[0]:.1f}, {end_point[1]:.1f}, {end_point[2]:.1f})")
        print(f"  Length: {z_line_length:.1f} Å")
    
    return lines_angular, lines_z

def create_density_profiles_all_lines(universe, lines, slice_width, bin_size, 
                                     water_resname, ion_resname):
    """
    Create 1D density profiles for all lines efficiently by looping through atoms only once.
    """
    # Select atom groups
    water = universe.select_atoms(f'resname {water_resname}')
    ions = universe.select_atoms(f'resname {ion_resname}')
    hed = universe.select_atoms('resname HED')
    mm2 = universe.select_atoms('resname MM2')
    polar = universe.select_atoms('name A1 or name A2')
    
    groups = {
        'Water': water,
        'Ions': ions,
        'HED': hed,
#        'MM2': mm2,
        'Oxygen': polar
    }
    
    # Create separate binning for each line based on its length
    all_densities = []
    all_bin_centers = []
    
    for i, line_info in enumerate(lines):
        line_length = line_info['length']
        n_bins = int(np.ceil(line_length / bin_size))
        bin_edges = np.linspace(0, line_length, n_bins + 1)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        line_densities = {name: np.zeros(n_bins) for name in groups.keys()}
        all_densities.append(line_densities)
        all_bin_centers.append(bin_centers)
    
    box_dims = universe.dimensions[:3]
    
    print("Analyzing all lines efficiently...")
    
    # Loop through each group
    for group_name, atoms in groups.items():
        if len(atoms) == 0:
            continue
            
        print(f"  Processing {group_name}: {len(atoms)} atoms")
        positions = atoms.positions
        
        # Process each atom once
        for pos in positions:
            # Try all possible periodic images of the atom
            for dx in [-1, 0, 1]:
                # Create periodic image
                image_pos = pos + np.array([dx * box_dims[0],0,0])
                
                # Check this atom against ALL lines
                for line_idx, line_info in enumerate(lines):
                        
                    direction = line_info['direction']
                    start = line_info['start']
                    line_length = line_info['length']
                    
                    # Vector from line start to atom image
                    to_atom = image_pos - start
                    
                    # Project onto line direction
                    projection_length = np.dot(to_atom, direction)
                    
                    # Check if projection is within line bounds
                    if 0 <= projection_length <= line_length:
                        # Calculate perpendicular distance to line
                        projected_on_line = projection_length * direction
                        perpendicular_vector = to_atom - projected_on_line
                        
                        # Calculate distance in xz-plane for both types
                        perp_dist = np.sqrt(perpendicular_vector[0]**2 + perpendicular_vector[2]**2)
                        
                        # Check if within slice width
                        if perp_dist <= slice_width / 2:
                            # Find which bin
                            bin_idx = int(projection_length / bin_size)
                            n_bins = len(all_bin_centers[line_idx])
                            if 0 <= bin_idx < n_bins:
                                all_densities[line_idx][group_name][bin_idx] += 1

    
    # Convert to density (atoms per nm³)
    box_height = box_dims[1]  # y-dimension (full cylinder height)
    
    for line_idx in range(len(lines)):
        bin_volume = (bin_size * slice_width * box_height) / 1000  # Convert Å³ to nm³
        for group_name in groups.keys():
            all_densities[line_idx][group_name] = all_densities[line_idx][group_name] / bin_volume
    
    return all_bin_centers, all_densities


In [93]:
structure = '../run/run/run30.pdb'
slice_width = 12.5
water_resname = 'W'
ion_resname = 'Ions'
extend_length = 7
smooth_sigma = 3
bin_size=1

print(f"Loading system: {structure}")
u = mda.Universe(structure)

# Print system info
print(f"Box dimensions: {u.dimensions[:3]} Å")
print(f"Total atoms: {len(u.atoms)}")

# Print all residue names
residue_names = set(u.atoms.residues.resnames)
print(f"Present residue names: {sorted(residue_names)}")

Loading system: ../run/run/run30.pdb
Box dimensions: [195.316 128.443 515.593] Å
Total atoms: 167463
Present residue names: ['HED', 'Ions', 'LLG', 'MM2', 'RLG', 'W']


In [ ]:
from md_styler import MDStyler

sty = MDStyler().apply()

coms = calculate_cylinder_coms(u)
lines_angular, lines_z = define_analysis_lines(coms, extend_length)

# Fixed data-rectangle horizontal figure (16:9) with styler defaults
fig, ax = sty.fig_horizontal()

# --- atom selections
water = u.select_atoms(f'resname {water_resname}')
ions = u.select_atoms(f'resname {ion_resname}')
hed = u.select_atoms('resname HED')
polar = u.select_atoms('name A1 or name A2')

# atom-type -> color (VMD-matched, matte)
atom_colors = {
    "Water": sty.get_color("box"),
    "Ions":  sty.get_color("green"),
    "HED":   sty.get_color("orange"),
    "Oxygen": sty.get_color("red")
}

line = lines_z[0]

# --- plot atoms (xz)
for atoms, label, size in [
    (water, "Water", 0.2),
    (ions,  "Ions",  0.3),
    (hed,   "HED",   0.2),
    (polar, "Oxygen", 0.3)
]:
    if len(atoms) > 0:
        pos = atoms.positions
        ax.scatter(
            pos[:, 2], pos[:, 0],
            c=atom_colors[label],
            alpha=1,
            s=size,
            label=label,
        )

# --- orange analysis ribbon limited to the z-window (200–450 Å)
z_min, z_max = 250.0, 450.0  # used again for the density plot

z1, x1 = line['start'][2], line['start'][0]
z2, x2 = line['end'][2],   line['end'][0]

# parametric line, clip to [z_min, z_max]
if z2 != z1:
    t_min = (z_min - z1) / (z2 - z1)
    t_max = (z_max - z1) / (z2 - z1)
    t0, t1 = sorted([t_min, t_max])
    t0 = max(0.0, min(1.0, t0))
    t1 = max(0.0, min(1.0, t1))

    z_start = z1 + t0 * (z2 - z1)
    z_end   = z1 + t1 * (z2 - z1)
    x_start = x1 + t0 * (x2 - x1)
    x_end   = x1 + t1 * (x2 - x1)

    ax.plot(
        [z_start, z_end],
        [x_start, x_end],
        color=sty.get_color("gray"),
        linewidth=slice_width / 2,
        alpha=0.8,
        solid_capstyle="round",
    )

ax.set_xlabel("z (Å)")
ax.set_ylabel("x (Å)")
ax.set_title("2D Density Map (XZ plane) with Analysis Line")
ax.set_aspect("equal", adjustable="box")
# match domain & orientation we’ll use for density profiles
ax.set_xlim(z_min, z_max)
ax.set_ylim(50, 150)
ax.invert_xaxis()

plt.show()


In [86]:
from md_styler import MDStyler

sty = MDStyler().apply()

print("\nAnalyzing z-axis lines...")
bin_centers_z, densities_z = create_density_profiles_all_lines(
    u, lines_z, slice_width, bin_size,
    water_resname, ion_resname
)

print("\nAveraging angular profiles...")
group_names = [name for name in densities_z[0].keys() if name != "MM2"]
print(group_names)



Analyzing z-axis lines...
Analyzing all lines efficiently...
  Processing Water: 71763 atoms


  Processing Ions: 3300 atoms
  Processing HED: 13200 atoms
  Processing Oxygen: 6600 atoms

Averaging angular profiles...
['Water', 'Ions', 'HED', 'Oxygen']


In [94]:
common_bins_z = bin_centers_z[2]
n_bins_z = len(common_bins_z)
print(n_bins_z)
# --- build symmetric average in z, correcting the normalization
mid_point = n_bins_z // 2
start_second_half = mid_point + (n_bins_z % 2)  # skip central bin if odd

averaged_z = {name: np.zeros(mid_point) for name in group_names}

for densities in densities_z:
    for group_name in group_names:
        # first half + mirrored second half
        averaged_z[group_name] += densities[group_name][0:mid_point]
        averaged_z[group_name] += densities[group_name][start_second_half:][::-1]

# divide by 2 * number of profiles (each contributes twice)
norm_factor = 2.0 * len(densities_z)
for group_name in group_names:
    averaged_z[group_name] /= norm_factor

# --- optional smoothing
smoothed_z = {}
for group_name in group_names:
    if np.sum(averaged_z[group_name]) > 0:
        smoothed_z[group_name] = gaussian_filter1d(
            averaged_z[group_name], sigma=smooth_sigma
        )
    else:
        smoothed_z[group_name] = averaged_z[group_name]


360


In [ ]:
# --- choose zoom region to MATCH scatter plot z-range
z_min, z_max = 250.0, 450.0

start_idx_z = 0
end_idx_z   = 179
zoom_bins_z = common_bins_z[start_idx_z:end_idx_z + 1]

# ======================
# PLOTTING (styler only)
# ======================

fig, ax = sty.fig_horizontal()
colors = sty.get_palette(len(group_names))

ax_twins = []

# determine y-limits in zoom region per group (unchanged)
zoom_y_limits = {name: 0.0 for name in group_names}
for group_name, density in smoothed_z.items():
    if np.sum(density) > 0:
        zoom_data = density[start_idx_z:end_idx_z + 1]
        zoom_y_limits[group_name] = 1.1 * np.max(zoom_data)

# map ribbon distance (0–160 Å) to scatter z-axis coordinates:
# densities start at z_start and go left to z_start - 160.
# zoom_bins_z is distance along the ribbon; we place it onto the z-axis.
x_density = z_start - zoom_bins_z

for i, (group_name, density) in enumerate(smoothed_z.items()):
    if np.sum(density) == 0:
        continue

    zoom_data = density[start_idx_z:end_idx_z + 1]
    color = atom_colors[group_name]

    if i == 0:
        ax_current = ax
        ax_current.set_ylabel(
            f"{group_name} density (atoms/nm³)", color=color
        )
        ax_current.tick_params(axis="y", labelcolor=color)
    else:
        ax_current = ax.twinx()
        ax_twins.append(ax_current)

        # use empty space (450 → z_start) for extra y-axes
        ax_current.spines["left"].set_position(("outward", 60 * i))
        ax_current.spines["right"].set_visible(False)
        ax_current.yaxis.set_label_position("left")
        ax_current.yaxis.tick_left()
        ax_current.set_ylabel(
            f"{group_name} density (atoms/nm³)", color=color
        )
        ax_current.tick_params(axis="y", labelcolor=color)

    # plot using mapped z-axis positions
    ax_current.plot(
        x_density,
        zoom_data,
        color=color,
        linewidth=2.0,
        label=group_name,
    )
    ax_current.set_ylim(0, zoom_y_limits[group_name])

# x-axis: EXACTLY same range and direction as the scatter plot
ax.set_xlim(z_max, z_min)      # 450 -> 200
ax.set_xlabel("z (Å)")         # matches scatter axis
# no need to invert here because limits are already given as (max, min)

# light y-grid
ax.grid(True, axis="y", alpha=0.3)

plt.show()


In [ ]:
from md_styler import MDStyler
import matplotlib.pyplot as plt

sty = MDStyler().apply()
coms = calculate_cylinder_coms(u)
lines_angular, lines_z = define_analysis_lines(coms, extend_length)

# Create figure with two subplots stacked vertically using styler's fixed axes
fig = plt.figure(figsize=(3.5, 6.0), facecolor='white', dpi=sty.dpi)

# Create two axes with same width, stacked vertically
# Using gridspec for better control - adjusted spacing to move top figure down
from matplotlib.gridspec import GridSpec
gs = GridSpec(2, 1, figure=fig, height_ratios=[1, 1.2], hspace=0.01,
              left=0.12, right=0.88, bottom=0.01, top=0.7)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)

# Apply styler defaults to both axes
sty._apply_axes_defaults(ax1)
sty._apply_axes_defaults(ax2)

# --- atom selections
water = u.select_atoms(f'resname {water_resname}')
ions = u.select_atoms(f'resname {ion_resname}')
hed = u.select_atoms('resname HED')
polar = u.select_atoms('name A1 or name A2')

# atom-type -> color (VMD-matched, matte)
atom_colors = {
    "Water": sty.get_color("box"),
    "Ions":  sty.get_color("green"),
    "HED":   sty.get_color("orange"),
    "Oxygen": sty.get_color("red")
}

line = lines_z[0]

# ======================
# FIRST SUBPLOT: Scatter plot
# ======================
z_min, z_max = 250.0, 450.0

# --- plot atoms (xz)
for atoms, label, size in [
    (water, "Water", 0.2),
    (ions,  "Ions",  0.3),
    (hed,   "HED",   0.2),
    (polar, "Oxygen", 0.3)
]:
    if len(atoms) > 0:
        pos = atoms.positions
        ax1.scatter(
            pos[:, 2], pos[:, 0],
            c=atom_colors[label],
            alpha=1,
            s=size,
            label=label,
        )

# --- orange analysis ribbon limited to the z-window (200–450 Å)
z1, x1 = line['start'][2], line['start'][0]
z2, x2 = line['end'][2],   line['end'][0]

# parametric line, clip to [z_min, z_max]
if z2 != z1:
    t_min = (z_min - z1) / (z2 - z1)
    t_max = (z_max - z1) / (z2 - z1)
    t0, t1 = sorted([t_min, t_max])
    t0 = max(0.0, min(1.0, t0))
    t1 = max(0.0, min(1.0, t1))
    z_start = z1 + t0 * (z2 - z1)
    z_end   = z1 + t1 * (z2 - z1)
    x_start = x1 + t0 * (x2 - x1)
    x_end   = x1 + t1 * (x2 - x1)
    ax1.plot(
        [z_start, z_end],
        [x_start, x_end],
        color=sty.get_color("gray"),
        linewidth=slice_width / 2,
        alpha=0.8,
        solid_capstyle="round",
        label="Analysis line"  # ADD THIS LINE
    )

# Remove x-axis label from top plot (shared x-axis)
ax1.tick_params(labelbottom=True)
ax1.set_ylabel("x (Å)")
ax1.set_title("2D Density Map (XZ plane) with Analysis Line")
ax1.set_aspect("equal", adjustable="box")
ax1.set_xlim(z_min, z_max)
ax1.set_ylim(50, 150)
ax1.invert_xaxis()

handles, labels = ax1.get_legend_handles_labels()
# Customize the label names here
label_map = {
    "Water": "Water",           # Change these to whatever you want
    "Ions": "Ca²⁺",             # Example: using superscript
    "HED": "Hydrophobic groups",        # Example: more descriptive
    "Oxygen": "Carboxylates",   # Example: chemical group name
    "Analysis line": "Profiling region"  # Example: more descriptive
}
labels = [label_map.get(l, l) for l in labels]

legend = ax1.legend(handles, labels, loc='upper left', bbox_to_anchor=(-0.78, 0.63), 
                    frameon=False, ncol=1, markerscale=7.0)

# ======================
# SECOND SUBPLOT: Density profiles
# ======================
start_idx_z = 0
end_idx_z   = 179
zoom_bins_z = common_bins_z[start_idx_z:end_idx_z + 1]

ax_twins = []

# determine y-limits in zoom region per group
zoom_y_limits = {name: 0.0 for name in group_names}
for group_name, density in smoothed_z.items():
    if np.sum(density) > 0:
        zoom_data = density[start_idx_z:end_idx_z + 1]
        zoom_y_limits[group_name] = 1.1 * np.max(zoom_data)

# map ribbon distance to z-axis coordinates
x_density = z_start - zoom_bins_z

# Label mapping for density plot y-axes
density_label_map = {
    "Water": "Water",
    "Ions": "Ca²⁺",
    "HED": "Hydrophobic",
    "Oxygen": "Carboxylates"
}

for i, (group_name, density) in enumerate(smoothed_z.items()):
    if np.sum(density) == 0:
        continue
    
    zoom_data = density[start_idx_z:end_idx_z + 1]
    color = atom_colors[group_name]
    
    # Get display name for y-axis label
    display_name = density_label_map.get(group_name, group_name)
    
    if i == 0:
        ax_current = ax2
        ax_current.set_ylabel(
            f"{display_name} density (atoms/nm³)", color=color
        )
        ax_current.tick_params(axis="y", labelcolor=color)
    else:
        ax_current = ax2.twinx()
        ax_twins.append(ax_current)
        ax_current.spines["left"].set_position(("outward", 38 * i))
        ax_current.spines["right"].set_visible(False)
        ax_current.yaxis.set_label_position("left")
        ax_current.yaxis.tick_left()
        ax_current.set_ylabel(
            f"{display_name} density (atoms/nm³)", color=color
        )
        ax_current.tick_params(axis="y", labelcolor=color)
    
    # plot using mapped z-axis positions
    ax_current.plot(
        x_density,
        zoom_data,
        color=color,
        linewidth=2.0,
        label=group_name,
    )
    ax_current.set_ylim(0, zoom_y_limits[group_name])

# x-axis: EXACTLY same range and direction as the scatter plot
ax2.set_xlim(z_max, z_min)
ax2.set_xlabel("z (Å)")
ax2.grid(True, axis="y", alpha=0.3)

plt.show()